# transformers-ipfs Example in Google Colab

This notebook demonstrates how to use Hugging Face Transformers with IPFS models using the transformers-ipfs integration.

## Step 1: Install Required Packages

In [ ]:
# Install transformers, PyTorch, and transformers-ipfs
!pip install transformers torch transformers-ipfs --quiet

## Step 2: Activate IPFS Integration

In most environments, you would use `transformers-ipfs activate`, but in Google Colab we need to apply the patch explicitly:

In [ ]:
# Standard activation approach:
# ~# transformers-ipfs activate

# Google Colab activation approach:
import transformers_ipfs; transformers_ipfs.activate()

## Step 3: Load and Use an IPFS-hosted Model

Now we can load models directly from IPFS using their CID:

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load model directly from IPFS
model_uri = "ipfs://bafybeichqdarufyutqc7yd43k77fkxbmeuhhetbihd3g32ghcqvijp6fxi"
# Equivalent HuggingFace model: model_uri = "riturajpandey739/gpt2-sentiment-analysis-tweets"

tokenizer = AutoTokenizer.from_pretrained(model_uri)
model = AutoModelForSequenceClassification.from_pretrained(model_uri)

LABELS = ["negative", "neutral", "positive"]

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    sentiment = LABELS[predicted_class]
    confidence = float(probabilities[0][predicted_class])
    return sentiment, confidence

example_tweets = [
    "I absolutely love this new update! The features are amazing! 😊",
    "This is the worst service I've ever experienced. Very disappointed. 😠",
    "The weather is nice today, perfect for a walk in the park!",
    "This product is not worth the money, don't buy it 👎"
]

for tweet in example_tweets:
    sentiment, confidence = analyze_sentiment(tweet)
    print(f"Tweet: {tweet}")
    print(f"Sentiment: {sentiment} (confidence: {confidence:.2%})\n")

## How It Works

The `transformers-ipfs` package patches the Hugging Face Transformers library to:

1. Recognize `ipfs://` URIs as valid model identifiers
2. Download model files from IPFS nodes or gateways
3. Cache models locally for faster loading in subsequent runs

This allows you to load models from a decentralized network without changing any of your existing code!